<a href="https://colab.research.google.com/github/FarrelAD/Hology-8-2025-Data-Mining-PRIVATE/blob/main/notebooks/scratch/nb_01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Crowd Counting Model

This notebook implements crowd counting models.

**Features:**
- Multiple model architectures (SimpleCountingNet and ImprovedCrowdCounter)
- Density map generation with Gaussian filtering
- Multi-task learning approach (density + count prediction)
- Advanced data augmentation and preprocessing
- Model evaluation and visualization
- Enhanced loss functions with multi-task optimization

# Import Libraries

In [ ]:
# Import Required Libraries and Setup
import os
import sys
import json
import math
import random
import warnings
from typing import List, Tuple, Dict, Any, Union, Optional
from datetime import datetime

import numpy as np
import pandas as pd
import cv2
import matplotlib.pyplot as plt
from PIL import Image
from tqdm import tqdm

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
import torchvision.models as models

# Suppress warnings
warnings.filterwarnings('ignore')

print("✅ All libraries imported successfully!")

# Detect environment
def detect_environment():
    """Detect if running in Colab, Kaggle, or local environment"""
    if 'google.colab' in sys.modules:
        return 'colab'
    elif 'kaggle_secrets' in sys.modules or os.environ.get('KAGGLE_KERNEL_RUN_TYPE'):
        return 'kaggle'
    else:
        return 'local'

ENV = detect_environment()
print(f"🔍 Detected environment: {ENV.upper()}")

# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"🔧 Using device: {device}")

if torch.cuda.is_available():
    print(f"CUDA device: {torch.cuda.get_device_name(0)}")
    print(f"CUDA memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")

# Dataset Setup and Configuration

In [ ]:
# Environment-specific dataset setup
def setup_dataset_paths(env: str) -> Dict[str, str]:
    """Setup dataset paths based on environment"""
    
    if env == 'colab':
        # Google Colab paths
        dataset_name = "penyisihan-hology-8-0-2025-data-mining"
        drive_path = "/content/drive/MyDrive/PROJECTS/Cognivio/dataset"
        local_path = "/content/dataset"
        
        # Mount Google Drive and setup Kaggle credentials
        from google.colab import drive
        drive.mount('/content/drive')
        
        # Setup dataset download logic here...
        
        return {
            'img_dir': f"{local_path}/train/images",
            'label_dir': f"{local_path}/train/labels", 
            'test_dir': f"{local_path}/test/images",
            'save_dir': "/content/drive/MyDrive/PROJECTS/Cognivio/models"
        }
    
    elif env == 'kaggle':
        # Kaggle paths
        return {
            'img_dir': "/kaggle/input/penyisihan-hology-8-0-2025-data-mining/train/images",
            'label_dir': "/kaggle/input/penyisihan-hology-8-0-2025-data-mining/train/labels",
            'test_dir': "/kaggle/input/penyisihan-hology-8-0-2025-data-mining/test/images",
            'save_dir': "/kaggle/working"
        }
    
    else:  # local
        # Local paths
        base_path = "../../data"
        return {
            'img_dir': f"{base_path}/train/images",
            'label_dir': f"{base_path}/train/labels",
            'test_dir': f"{base_path}/test/images",
            'save_dir': r"d:\Hology\Hology-8-2025-Data-Mining-PRIVATE"
        }

# Setup paths
paths = setup_dataset_paths(ENV)
print(f"📁 Dataset paths configured for {ENV}:")
for key, path in paths.items():
    exists = "✅" if os.path.exists(path) else "⚠️"
    print(f"   {key}: {path} {exists}")

# Create save directory
os.makedirs(paths['save_dir'], exist_ok=True)

# Training Configuration

In [ ]:
# Training Configuration
config = {
    # Data paths
    'img_dir': paths['img_dir'],
    'label_dir': paths['label_dir'],
    'test_dir': paths['test_dir'],
    'save_dir': paths['save_dir'],
    
    # Model parameters
    'target_size': (512, 512),
    'batch_size': 4,
    'epochs': 50,
    'lr': 1e-4,
    'weight_decay': 1e-4,
    
    # Training parameters
    'num_workers': 4,
    'pin_memory': True,
    'early_stop_patience': 5,
    
    # Model saving
    'simple_model_path': os.path.join(paths['save_dir'], 'best_simple_model.pth'),
    'improved_model_path': os.path.join(paths['save_dir'], 'improved_counting_best.pth'),
    'submission_path': os.path.join(paths['save_dir'], 'submission_improved_model.csv'),
    'seed': 42
}

print("📋 Configuration loaded:")
for key, value in config.items():
    print(f"  {key}: {value}")

# Set random seed for reproducibility

In [ ]:
def set_seed(seed: int = 42) -> None:
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

set_seed(config['seed'])
print(f"🎲 Random seed set to {config['seed']}")

# Data Analysis and Exploration

In [ ]:
print(f"🔍 Analyzing dataset...")

# Check dataset sizes
train_images = os.listdir(config['img_dir'])
train_labels = os.listdir(config['label_dir'])
test_images = os.listdir(config['test_dir'])

print(f"\n📊 Dataset Overview:")
print(f"Train images: {len(train_images)}")
print(f"Train labels: {len(train_labels)}")
print(f"Test images: {len(test_images)}")

# Load and examine sample labels
print(f"\n🔍 Sample label examination:")
sample_labels = []
for i, label_file in enumerate(train_labels[:3]):
    with open(os.path.join(config['label_dir'], label_file), 'r') as f:
        label_data = json.load(f)
        sample_labels.append(label_data)
        print(f"Sample {i+1}: {label_file}")
        print(f"  Image ID: {label_data['img_id']}")
        print(f"  Human count: {label_data['human_num']}")
        print(f"  Number of points: {len(label_data['points'])}")

# Analyze crowd count distribution
crowd_counts = []
for label_file in train_labels:
    with open(os.path.join(config['label_dir'], label_file), 'r') as f:
        label_data = json.load(f)
        crowd_counts.append(label_data['human_num'])

crowd_counts = np.array(crowd_counts)
print(f"\n📈 Crowd count statistics:")
print(f"Mean: {crowd_counts.mean():.2f}")
print(f"Std: {crowd_counts.std():.2f}")
print(f"Min: {crowd_counts.min()}")
print(f"Max: {crowd_counts.max()}")
print(f"Median: {np.median(crowd_counts):.2f}")

# Visualize distribution and sample images

In [ ]:
plt.figure(figsize=(12, 8))

plt.subplot(2, 2, 1)
plt.hist(crowd_counts, bins=50, alpha=0.7, edgecolor='black')
plt.title('Distribution of Crowd Counts')
plt.xlabel('Number of People')
plt.ylabel('Frequency')

plt.subplot(2, 2, 2)
plt.boxplot(crowd_counts)
plt.title('Crowd Count Box Plot')
plt.ylabel('Number of People')

# Load and visualize sample images with annotations
sample_img_path = os.path.join(config['img_dir'], "1.jpg")
sample_label_path = os.path.join(config['label_dir'], "1.json")

# Load image
img = cv2.imread(sample_img_path)
img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

# Load annotations
with open(sample_label_path, 'r') as f:
    annotations = json.load(f)

plt.subplot(2, 2, 3)
plt.imshow(img_rgb)
# Plot annotation points
points = annotations['points']
x_coords = [p['x'] for p in points]
y_coords = [p['y'] for p in points]
plt.scatter(x_coords, y_coords, c='red', s=10, alpha=0.6)
plt.title(f'Sample Image: {annotations["human_num"]} people')
plt.axis('off')

# Visualize another sample
sample_img_path2 = os.path.join(config['img_dir'], "100.jpg")
sample_label_path2 = os.path.join(config['label_dir'], "100.json")

img2 = cv2.imread(sample_img_path2)
img2_rgb = cv2.cvtColor(img2, cv2.COLOR_BGR2RGB)

with open(sample_label_path2, 'r') as f:
    annotations2 = json.load(f)

plt.subplot(2, 2, 4)
plt.imshow(img2_rgb)
points2 = annotations2['points']
x_coords2 = [p['x'] for p in points2]
y_coords2 = [p['y'] for p in points2]
plt.scatter(x_coords2, y_coords2, c='red', s=15, alpha=0.8)
plt.title(f'Sample Image: {annotations2["human_num"]} people')
plt.axis('off')

plt.tight_layout()
plt.show()

# Dataset Implementation

In [ ]:
class CrowdCountingDataset(Dataset):
    """Custom dataset for crowd counting with density map generation.
    
    Supports density map generation using Gaussian filtering and various
    data augmentation options.
    """
    
    def __init__(
        self,
        images_dir: str,
        labels_dir: str,
        transform: Optional[Any] = None,
        target_size: Tuple[int, int] = (512, 512)
    ) -> None:
        self.images_dir = images_dir
        self.labels_dir = labels_dir
        self.transform = transform
        self.target_size = target_size
        
        # Get all image files
        self.image_files = [f for f in os.listdir(images_dir) if f.endswith('.jpg')]
        self.image_files.sort()
        
    def __len__(self) -> int:
        return len(self.image_files)
    
    def __getitem__(self, idx: int) -> Tuple[torch.Tensor, torch.Tensor, torch.Tensor]:
        # Load image
        img_name = self.image_files[idx]
        img_path = os.path.join(self.images_dir, img_name)
        image = cv2.imread(img_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
        # Load annotations
        label_name = img_name.replace('.jpg', '.json')
        label_path = os.path.join(self.labels_dir, label_name)
        
        with open(label_path, 'r') as f:
            label_data = json.load(f)
        
        # Get original image dimensions
        orig_h, orig_w = image.shape[:2]
        
        # Resize image
        image = cv2.resize(image, self.target_size)
        
        # Create density map
        density_map = self.create_density_map(
            label_data['points'], orig_w, orig_h, self.target_size
        )
        
        # Apply transforms if any
        if self.transform:
            image = self.transform(image)
        else:
            # Convert to tensor and normalize
            image = torch.from_numpy(image).permute(2, 0, 1).float() / 255.0
        
        density_map = torch.from_numpy(density_map).float()
        count = torch.tensor(label_data['human_num'], dtype=torch.float32)
        
        return image, density_map, count
    
    def create_density_map(
        self,
        points: List[Dict[str, float]],
        orig_w: int,
        orig_h: int,
        target_size: Tuple[int, int]
    ) -> np.ndarray:
        """Create Gaussian density map from point annotations"""
        # Scale factor for resizing
        scale_x = target_size[0] / orig_w
        scale_y = target_size[1] / orig_h
        
        # Initialize density map
        density_map = np.zeros(target_size[::-1], dtype=np.float32)  # (height, width)
        
        if len(points) == 0:
            return density_map
        
        # Scale points to new dimensions
        scaled_points = []
        for point in points:
            x_scaled = point['x'] * scale_x
            y_scaled = point['y'] * scale_y
            
            # Ensure points are within bounds
            x_scaled = max(0, min(target_size[0] - 1, x_scaled))
            y_scaled = max(0, min(target_size[1] - 1, y_scaled))
            
            scaled_points.append((int(x_scaled), int(y_scaled)))
        
        # Create Gaussian kernels for each point
        sigma = 4.0  # Standard deviation for Gaussian kernel
        kernel_size = int(6 * sigma)  # Kernel size (6 sigma rule)
        
        for x, y in scaled_points:
            # Create Gaussian kernel
            y_min = max(0, y - kernel_size)
            y_max = min(target_size[1], y + kernel_size + 1)
            x_min = max(0, x - kernel_size)
            x_max = min(target_size[0], x + kernel_size + 1)
            
            # Generate mesh grid for the region
            yy, xx = np.meshgrid(range(y_min, y_max), range(x_min, x_max), indexing='ij')
            
            # Calculate Gaussian values
            gaussian = np.exp(-((xx - x) ** 2 + (yy - y) ** 2) / (2 * sigma ** 2))
            
            # Add to density map
            density_map[y_min:y_max, x_min:x_max] += gaussian
        
        return density_map

print("✅ CrowdCountingDataset class implemented!")

# Model Architectures

In [ ]:
# Utility function to count parameters
def count_parameters(model: nn.Module) -> int:
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

# Simple Model Definition
class SimpleCountingNet(nn.Module):
    """Simple CNN-based crowd counting model with dual outputs."""
    
    def __init__(self) -> None:
        super(SimpleCountingNet, self).__init__()
        
        # Simple CNN backbone
        self.features = nn.Sequential(
            nn.Conv2d(3, 64, 3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2, 2),
            
            nn.Conv2d(64, 128, 3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2, 2),
            
            nn.Conv2d(128, 256, 3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2, 2),
            
            nn.Conv2d(256, 512, 3, padding=1),
            nn.ReLU(inplace=True),
        )
        
        # Density regression head
        self.density_head = nn.Sequential(
            nn.Conv2d(512, 256, 3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 128, 3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(128, 1, 1),
            nn.ReLU()
        )
        
        # Count regression head
        self.count_head = nn.Sequential(
            nn.AdaptiveAvgPool2d((1, 1)),
            nn.Flatten(),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(256, 1),
            nn.ReLU()
        )
        
    def forward(self, x: torch.Tensor) -> Tuple[torch.Tensor, torch.Tensor]:
        # Extract features
        features = self.features(x)
        
        # Density map prediction
        density = self.density_head(features)
        # Upsample to quarter resolution
        density = F.interpolate(density, scale_factor=8, mode='bilinear', align_corners=False)
        
        # Count prediction
        count = self.count_head(features).squeeze()
        
        return density, count

# Improved model with better architecture
class ImprovedCrowdCounter(nn.Module):
    """Advanced crowd counting model with ResNet-like architecture."""
    
    def __init__(self) -> None:
        super(ImprovedCrowdCounter, self).__init__()
        
        # Use ResNet-like blocks
        self.conv1 = nn.Sequential(
            nn.Conv2d(3, 64, 7, stride=2, padding=3),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(3, stride=2, padding=1)
        )
        
        # Residual blocks
        self.layer1 = self._make_layer(64, 128, 2, stride=1)
        self.layer2 = self._make_layer(128, 256, 2, stride=2)
        self.layer3 = self._make_layer(256, 512, 2, stride=2)
        
        # Density estimation branch
        self.density_branch = nn.Sequential(
            nn.Conv2d(512, 256, 3, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 128, 3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.Conv2d(128, 64, 3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.Conv2d(64, 1, 1),
            nn.ReLU()
        )
        
        # Count estimation branch (global)
        self.count_branch = nn.Sequential(
            nn.AdaptiveAvgPool2d((1, 1)),
            nn.Flatten(),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(128, 1),
            nn.ReLU()
        )
        
        self._initialize_weights()
    
    def _make_layer(
        self,
        in_channels: int,
        out_channels: int,
        num_blocks: int,
        stride: int = 1
    ) -> nn.Sequential:
        layers = []
        # First block (may downsample)
        layers.append(nn.Conv2d(in_channels, out_channels, 3, stride=stride, padding=1))
        layers.append(nn.BatchNorm2d(out_channels))
        layers.append(nn.ReLU(inplace=True))
        
        # Remaining blocks
        for _ in range(1, num_blocks):
            layers.append(nn.Conv2d(out_channels, out_channels, 3, padding=1))
            layers.append(nn.BatchNorm2d(out_channels))
            layers.append(nn.ReLU(inplace=True))
        
        return nn.Sequential(*layers)
    
    def forward(self, x: torch.Tensor) -> Tuple[torch.Tensor, torch.Tensor]:
        # Feature extraction
        x = self.conv1(x)
        x = self.layer1(x)
        x = self.layer2(x)
        features = self.layer3(x)
        
        # Density estimation
        density = self.density_branch(features)
        
        # Upsample density to quarter resolution
        density = F.interpolate(density, scale_factor=4, mode='bilinear', align_corners=False)
        
        # Count estimation
        count = self.count_branch(features).squeeze()
        
        return density, count
    
    def _initialize_weights(self) -> None:
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.Linear):
                nn.init.normal_(m.weight, 0, 0.01)
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)

print("✅ Model architectures implemented!")

# Test model instantiation
print("🔍 Testing model architectures...")
simple_test = SimpleCountingNet().to(device)
improved_test = ImprovedCrowdCounter().to(device)

print(f"📊 SimpleCountingNet parameters: {count_parameters(simple_test):,}")
print(f"📊 ImprovedCrowdCounter parameters: {count_parameters(improved_test):,}")

del simple_test, improved_test

# Loss Functions

In [ ]:
# Basic CrowdCountingLoss for backward compatibility 
class CrowdCountingLoss(nn.Module):
    """Multi-task loss for crowd counting with density and count predictions."""
    
    def __init__(self, alpha: float = 1.0, beta: float = 1.0) -> None:
        super(CrowdCountingLoss, self).__init__()
        self.alpha = alpha  # density loss weight
        self.beta = beta    # count loss weight
        
        self.mse = nn.MSELoss()
        
    def forward(
        self,
        density_pred: torch.Tensor,
        count_pred: torch.Tensor,
        density_target: torch.Tensor,
        count_target: torch.Tensor
    ) -> torch.Tensor:
        # Density map loss
        density_loss = self.mse(density_pred, density_target)
        
        # Count loss  
        count_loss = self.mse(count_pred, count_target.float())
        
        # Combined loss
        total_loss = self.alpha * density_loss + self.beta * count_loss
        
        return total_loss

# Multi-task loss for both density and count
class MultiTaskLoss(nn.Module):
    """Enhanced multi-task loss with separate loss components."""
    
    def __init__(self, alpha: float = 1.0, beta: float = 1.0) -> None:
        super(MultiTaskLoss, self).__init__()
        self.alpha = alpha
        self.beta = beta
        self.mse = nn.MSELoss()
    
    def forward(
        self,
        pred_density: torch.Tensor,
        true_density: torch.Tensor,
        pred_count: torch.Tensor,
        true_count: torch.Tensor
    ) -> Tuple[torch.Tensor, torch.Tensor, torch.Tensor]:
        density_loss = self.mse(pred_density, true_density)
        count_loss = self.mse(pred_count, true_count)
        
        total_loss = self.alpha * density_loss + self.beta * count_loss
        return total_loss, density_loss, count_loss

print("✅ Loss functions implemented!")

# Data Loading and Preparation

In [ ]:
print("🔍 Preparing datasets...")

# Data transforms
train_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Create datasets
full_dataset = CrowdCountingDataset(
    config['img_dir'],
    config['label_dir'],
    target_size=config['target_size']
)

# Split dataset (80% train, 20% validation)
dataset_size = len(full_dataset)
train_size = int(0.8 * dataset_size)
val_size = dataset_size - train_size

train_dataset, val_dataset = torch.utils.data.random_split(
    full_dataset, [train_size, val_size], 
    generator=torch.Generator().manual_seed(config['seed'])
)

print(f"📦 Dataset sizes: Train={len(train_dataset)}, Val={len(val_dataset)}")

# Create data loaders
train_loader = DataLoader(
    train_dataset,
    batch_size=config['batch_size'],
    shuffle=True,
    num_workers=config['num_workers'],
    pin_memory=config['pin_memory'],
    drop_last=True
)

val_loader = DataLoader(
    val_dataset,
    batch_size=config['batch_size'],
    shuffle=False,
    num_workers=config['num_workers'],
    pin_memory=config['pin_memory']
)

print(f"🚀 Data loaders created!")
print(f"   Train batches: {len(train_loader)}")
print(f"   Val batches: {len(val_loader)}")

# Test the dataset
sample_img, sample_density, sample_count = full_dataset[0]
print(f"📊 Sample data shapes:")
print(f"   Image: {sample_img.shape}")
print(f"   Density map: {sample_density.shape}")
print(f"   Count: {sample_count}, Density sum: {sample_density.sum():.2f}")

# Training Functions

In [ ]:
def train_model_improved(
    model: nn.Module,
    train_loader: DataLoader,
    val_loader: DataLoader,
    epochs: int = 15,
    lr: float = 0.001,
    weight_decay: float = 1e-4,
    save_path: str = 'best_model.pth'
) -> Tuple[List[float], List[float], List[float], List[float], float]:
    """
    Improved training with better hyperparameters and early stopping
    """
    model = model.to(device)
    
    # Use AdamW optimizer with weight decay
    optimizer = torch.optim.AdamW(model.parameters(), lr=lr, weight_decay=weight_decay)
    
    # Learning rate scheduler
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=epochs)
    
    # Loss function
    criterion = CrowdCountingLoss(alpha=1.0, beta=0.01)  
    
    # Training history
    train_losses, val_losses = [], []
    train_maes, val_maes = [], []
    
    best_mae = float('inf')
    patience = 0
    patience_limit = config['early_stop_patience']
    
    print(f"🚀 Starting training for {epochs} epochs...")
    
    for epoch in range(epochs):
        print(f"\n📈 Epoch {epoch+1}/{epochs}")
        
        # Training phase
        model.train()
        running_train_loss = 0.0
        running_train_mae = 0.0
        
        for batch_idx, (images, density_maps, counts) in enumerate(tqdm(train_loader, desc="Train", leave=False)):
            images = images.to(device)
            density_maps = density_maps.to(device)  
            counts = counts.to(device)
            
            optimizer.zero_grad()
            
            pred_density, pred_counts = model(images)
            
            loss = criterion(pred_density, pred_counts, density_maps, counts)
            
            loss.backward()
            optimizer.step()
            
            running_train_loss += loss.item()
            
            # Calculate MAE for count prediction
            mae = torch.mean(torch.abs(pred_counts - counts)).item()
            running_train_mae += mae
        
        # Validation phase
        model.eval()
        running_val_loss = 0.0
        running_val_mae = 0.0
        
        with torch.no_grad():
            for images, density_maps, counts in tqdm(val_loader, desc="Val", leave=False):
                images = images.to(device)
                density_maps = density_maps.to(device)
                counts = counts.to(device)
                
                pred_density, pred_counts = model(images)
                loss = criterion(pred_density, pred_counts, density_maps, counts)
                
                running_val_loss += loss.item()
                mae = torch.mean(torch.abs(pred_counts - counts)).item()
                running_val_mae += mae
        
        # Calculate epoch metrics
        epoch_train_loss = running_train_loss / len(train_loader)
        epoch_val_loss = running_val_loss / len(val_loader)
        epoch_train_mae = running_train_mae / len(train_loader)
        epoch_val_mae = running_val_mae / len(val_loader)
        
        train_losses.append(epoch_train_loss)
        val_losses.append(epoch_val_loss)
        train_maes.append(epoch_train_mae)
        val_maes.append(epoch_val_mae)
        
        # Update learning rate
        scheduler.step()
        
        print(f"📊 Train Loss: {epoch_train_loss:.4f}, Train MAE: {epoch_train_mae:.2f}")
        print(f"📊 Val Loss: {epoch_val_loss:.4f}, Val MAE: {epoch_val_mae:.2f}")
        print(f"🎯 Learning Rate: {optimizer.param_groups[0]['lr']:.6f}")
        
        # Save best model based on validation MAE
        if epoch_val_mae < best_mae:
            best_mae = epoch_val_mae
            patience = 0
            torch.save(model.state_dict(), save_path)
            print(f"✅ New best model saved! MAE: {best_mae:.2f}")
        else:
            patience += 1
            print(f"⏳ No improvement for {patience} epoch(s)")
    
    return train_losses, val_losses, train_maes, val_maes, best_mae

def train_multitask_model(
    model: nn.Module,
    train_loader: DataLoader,
    val_loader: DataLoader,
    epochs: int = 20,
    lr: float = 1e-4
) -> Tuple[List[float], List[float], List[float], List[float], float]:
    """Improved training function for multi-task model"""
    
    criterion = MultiTaskLoss(alpha=0.1, beta=1.0)  # Emphasize count loss
    optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=1e-4)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
        optimizer, 'min', patience=5, factor=0.5, verbose=True
    )
    
    best_mae = float('inf')
    train_losses, val_losses = [], []
    train_maes, val_maes = [], []
    
    print(f"🚀 Starting multi-task training for {epochs} epochs...")
    
    for epoch in range(epochs):
        print(f"\n📈 Epoch {epoch+1}/{epochs}")
        print("-" * 40)
        
        # Training
        model.train()
        epoch_loss = 0
        epoch_mae = 0
        
        pbar = tqdm(train_loader, desc='Training')
        for images, density_maps, counts in pbar:
            images = images.to(device)
            density_maps = density_maps.to(device).unsqueeze(1)
            counts = counts.to(device)
            
            optimizer.zero_grad()
            
            # Forward
            pred_density, pred_count = model(images)
            
            # Resize density map if needed
            if pred_density.shape != density_maps.shape:
                pred_density = F.interpolate(
                    pred_density, size=density_maps.shape[2:], 
                    mode='bilinear', align_corners=False
                )
            
            # Loss calculation
            loss, density_loss, count_loss = criterion(
                pred_density, density_maps, pred_count, counts
            )
            
            # Backward
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            optimizer.step()
            
            # Metrics
            mae = torch.abs(pred_count - counts).mean().item()
            epoch_loss += loss.item()
            epoch_mae += mae
            
            pbar.set_postfix({
                'Loss': f'{loss.item():.4f}', 
                'MAE': f'{mae:.2f}', 
                'D_Loss': f'{density_loss.item():.4f}',
                'C_Loss': f'{count_loss.item():.4f}'
            })
        
        avg_train_loss = epoch_loss / len(train_loader)
        avg_train_mae = epoch_mae / len(train_loader)
        
        # Validation
        model.eval()
        val_loss = 0
        val_mae = 0
        val_mse = 0
        
        with torch.no_grad():
            for images, density_maps, counts in val_loader:
                images = images.to(device)
                density_maps = density_maps.to(device).unsqueeze(1)
                counts = counts.to(device)
                
                pred_density, pred_count = model(images)
                
                if pred_density.shape != density_maps.shape:
                    pred_density = F.interpolate(
                        pred_density, size=density_maps.shape[2:], 
                        mode='bilinear', align_corners=False
                    )
                
                loss, _, _ = criterion(pred_density, density_maps, pred_count, counts)
                mae = torch.abs(pred_count - counts).mean().item()
                mse = ((pred_count - counts) ** 2).mean().item()
                
                val_loss += loss.item()
                val_mae += mae
                val_mse += mse
        
        avg_val_loss = val_loss / len(val_loader)
        avg_val_mae = val_mae / len(val_loader)
        val_rmse = np.sqrt(val_mse / len(val_loader))
        
        # Scheduler step
        scheduler.step(avg_val_loss)
        
        # Store metrics
        train_losses.append(avg_train_loss)
        val_losses.append(avg_val_loss)
        train_maes.append(avg_train_mae)
        val_maes.append(avg_val_mae)
        
        print(f"📊 Train Loss: {avg_train_loss:.4f}, Train MAE: {avg_train_mae:.2f}")
        print(f"📊 Val Loss: {avg_val_loss:.4f}, Val MAE: {avg_val_mae:.2f}, Val RMSE: {val_rmse:.2f}")
        
        # Save best model
        if avg_val_mae < best_mae:
            best_mae = avg_val_mae
            torch.save({
                'epoch': epoch,
                'model_state_dict': model.state_dict(),
                'best_mae': best_mae,
            }, config['improved_model_path'])
            print(f"✅ New best model saved! MAE: {best_mae:.2f}")
    
    return train_losses, val_losses, train_maes, val_maes, best_mae

print("✅ Training functions implemented!")

## Training - Simple Model

### Initialize Simple Model

In [ ]:
print("🏗️ Training Simple Model...")

# Initialize simple model
simple_model = SimpleCountingNet().to(device)
print(f"📊 Simple model parameters: {count_parameters(simple_model):,}")

### Train the simple model

In [ ]:
print("🚀 Starting simple model training...")
simple_train_losses, simple_val_losses, simple_train_maes, simple_val_maes, simple_best_mae = train_model_improved(
    simple_model, train_loader, val_loader, 
    epochs=15, lr=0.001, weight_decay=1e-4,
    save_path=config['simple_model_path']
)

### Results Visualization and Analysis

In [ ]:
print(f"✅ Simple Model Training Complete - Best MAE: {simple_best_mae:.2f}")

# Plot simple model training curves
plt.figure(figsize=(15, 5))

plt.subplot(1, 3, 1)
plt.plot(simple_train_losses, label='Training Loss')
plt.plot(simple_val_losses, label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Simple Model: Training vs Validation Loss')
plt.legend()
plt.grid(True, alpha=0.3)

plt.subplot(1, 3, 2)
plt.plot(simple_train_maes, label='Training MAE')
plt.plot(simple_val_maes, label='Validation MAE')
plt.xlabel('Epoch')
plt.ylabel('MAE')
plt.title('Simple Model: Training vs Validation MAE')
plt.legend()
plt.grid(True, alpha=0.3)

plt.subplot(1, 3, 3)
plt.plot(simple_val_maes)
plt.xlabel('Epoch')
plt.ylabel('Validation MAE')
plt.title('Simple Model: Validation MAE Progress')
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## Training - Improved Model

### Initialize improved model

In [ ]:
print("🏗️ Training Improved Model...")

improved_model = ImprovedCrowdCounter().to(device)
print(f"📊 Improved model parameters: {count_parameters(improved_model):,}")

### Train the improved model

In [ ]:
print("🚀 Starting improved model training...")
improved_train_losses, improved_val_losses, improved_train_maes, improved_val_maes, improved_best_mae = train_multitask_model(
    improved_model, train_loader, val_loader, epochs=20, lr=1e-4
)

print(f"✅ Improved Model Training Complete - Best MAE: {improved_best_mae:.2f}")

### Results Visualization and Analysis

In [ ]:
# Plot improved model training curves
plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
plt.plot(improved_train_losses, label='Train Loss')
plt.plot(improved_val_losses, label='Val Loss')
plt.title('Improved Model: Training and Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.grid(True, alpha=0.3)
plt.yscale('log')

plt.subplot(1, 2, 2)
plt.plot(improved_train_maes, label='Train MAE')
plt.plot(improved_val_maes, label='Val MAE')
plt.title('Improved Model: Training and Validation MAE')
plt.xlabel('Epoch')
plt.ylabel('MAE')
plt.legend()
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Load best improved model for evaluation

In [ ]:
best_model_path = config['improved_model_path']
if os.path.exists(best_model_path):
    checkpoint = torch.load(best_model_path, map_location=device)
    improved_model.load_state_dict(checkpoint['model_state_dict'])
    print(f"✅ Loaded best model with MAE: {checkpoint['best_mae']:.2f}")

# Model Evaluation and Visualization

In [ ]:
def visualize_improved_predictions(
    model: nn.Module,
    dataset: Dataset,
    num_samples: int = 4
) -> None:
    """Visualize model predictions with density maps."""
    
    model.eval()
    
    fig, axes = plt.subplots(3, num_samples, figsize=(4*num_samples, 12))
    
    indices = random.sample(range(len(dataset)), num_samples)
    
    with torch.no_grad():
        for i, idx in enumerate(indices):
            image, true_density, true_count = dataset[idx]
            
            image_batch = image.unsqueeze(0).to(device)
            pred_density, pred_count = model(image_batch)
            
            if pred_density.shape[2:] != true_density.shape:
                pred_density = F.interpolate(
                    pred_density, size=true_density.shape, 
                    mode='bilinear', align_corners=False
                )
            
            image_np = image.permute(1, 2, 0).cpu().numpy()
            true_density_np = true_density.cpu().numpy()
            pred_density_np = pred_density.squeeze().cpu().numpy()
            pred_count_val = pred_count.item()
            
            # Original image
            axes[0, i].imshow(image_np)
            axes[0, i].set_title(f'Original\nTrue: {true_count:.0f}')
            axes[0, i].axis('off')
            
            # True density
            im1 = axes[1, i].imshow(true_density_np, cmap='jet')
            axes[1, i].set_title(f'True Density\nSum: {true_density_np.sum():.0f}')
            axes[1, i].axis('off')
            plt.colorbar(im1, ax=axes[1, i], fraction=0.046, pad=0.04)
            
            # Predicted density
            im2 = axes[2, i].imshow(pred_density_np, cmap='jet')
            axes[2, i].set_title(f'Predicted\nCount: {pred_count_val:.0f}')
            axes[2, i].axis('off')
            plt.colorbar(im2, ax=axes[2, i], fraction=0.046, pad=0.04)
    
    plt.tight_layout()
    plt.show()

print("🎨 Visualizing improved model predictions...")
visualize_improved_predictions(improved_model, val_dataset, num_samples=4)

# Test Prediction and Generate Submission

In [ ]:
def create_improved_predictions(
    model: nn.Module,
    test_images_path: str,
    test_image_names: List[str],
    device: str = 'cuda'
) -> List[int]:
    """Create predictions for test set with improved model"""
    
    model.eval()
    predictions = []
    
    print("📝 Creating predictions with improved model...")
    
    for image_name in tqdm(test_image_names, desc="Predicting"):
        image_path = os.path.join(test_images_path, image_name)
        
        if os.path.exists(image_path):
            # Load image
            image = cv2.imread(image_path)
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            image = cv2.resize(image, config['target_size'])
            
            # Convert to tensor
            image_tensor = torch.from_numpy(image).permute(2, 0, 1).float() / 255.0
            image_tensor = image_tensor.unsqueeze(0).to(device)
            
            with torch.no_grad():
                _, pred_count = model(image_tensor)
                predicted_count = max(0, int(round(pred_count.item())))
                predictions.append(predicted_count)
        else:
            print(f"⚠️ Warning: {image_name} not found")
            predictions.append(0)
    
    return predictions

# Create submission DataFrame structure
submission_df = pd.DataFrame({
    'image_id': [int(os.path.splitext(f)[0]) for f in test_images],
    'predicted_count': [0] * len(test_images)  # Placeholder
})
submission_df = submission_df.sort_values('image_id').reset_index(drop=True)

# Generate final predictions
final_predictions = create_improved_predictions(
    improved_model, config['test_dir'], test_images, device=device
)

# Update submission
submission_df['predicted_count'] = final_predictions

print("📋 Final predictions sample:")
print(submission_df.head(10))

print(f"\n📈 Final prediction statistics:")
print(f"Mean: {np.mean(final_predictions):.2f}")
print(f"Std: {np.std(final_predictions):.2f}")
print(f"Min: {np.min(final_predictions)}")
print(f"Max: {np.max(final_predictions)}")

# Save final submission
submission_df.to_csv(config['submission_path'], index=False)
print(f"✅ Final predictions saved to: {config['submission_path']}")

# Plot final prediction distribution
plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
plt.hist(final_predictions, bins=30, alpha=0.7, edgecolor='black')
plt.title('Distribution of Final Test Predictions')
plt.xlabel('Predicted Count')
plt.ylabel('Frequency')
plt.grid(True, alpha=0.3)

plt.subplot(1, 2, 2)
plt.boxplot(final_predictions)
plt.title('Final Test Predictions Box Plot')
plt.ylabel('Predicted Count')
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Model Saving and Summary

In [ ]:
# Save model metadata
model_metadata = {
    "models_trained": ["SimpleCountingNet", "ImprovedCrowdCounter"],
    "simple_model": {
        "architecture": "Simple CNN with dual outputs",
        "best_validation_mae": float(simple_best_mae),
        "total_parameters": count_parameters(simple_model),
        "save_path": config['simple_model_path']
    },
    "improved_model": {
        "architecture": "Multi-task ResNet-like with density + count estimation",
        "best_validation_mae": float(improved_best_mae),
        "total_parameters": count_parameters(improved_model),
        "save_path": config['improved_model_path']
    },
    "dataset_info": {
        "training_images": len(train_dataset),
        "validation_images": len(val_dataset),
        "test_images": len(test_images)
    },
    "final_predictions": {
        "range": [int(np.min(final_predictions)), int(np.max(final_predictions))],
        "mean": float(np.mean(final_predictions)),
        "submission_path": config['submission_path']
    },
    "created_date": datetime.now().isoformat(),
    "use_case": "Festival Harmoni Nusantara - Crowd Monitoring"
}

# Save metadata
metadata_path = os.path.join(config['save_dir'], 'model_metadata.json')
with open(metadata_path, 'w') as f:
    json.dump(model_metadata, f, indent=4)

print("\n" + "="*60)
print("🎉 CROWD COUNTING MODEL TRAINING COMPLETE!")
print("="*60)
print(f"✅ Dataset: {len(train_dataset)} train, {len(val_dataset)} val, {len(test_images)} test images")
print(f"✅ Simple Model - Best validation MAE: {simple_best_mae:.2f}")
print(f"✅ Improved Model - Best validation MAE: {improved_best_mae:.2f}")
print(f"✅ Final predictions range: {np.min(final_predictions)} - {np.max(final_predictions)} people")
print(f"✅ Average predicted count: {np.mean(final_predictions):.1f} people")
print(f"✅ Submission file: {os.path.basename(config['submission_path'])}")
print(f"✅ Model metadata saved to: {os.path.basename(metadata_path)}")
print("="*60)
print("🚀 Ready for Festival Harmoni Nusantara deployment!")

print("\n📁 FILES GENERATED:")
print("="*50)
print(f"✓ Simple Model: {os.path.basename(config['simple_model_path'])}")
print(f"✓ Improved Model: {os.path.basename(config['improved_model_path'])}")
print(f"✓ Final Predictions: {os.path.basename(config['submission_path'])}")
print(f"✓ Model Metadata: {os.path.basename(metadata_path)}")

# Validation check
submission_check = pd.read_csv(config['submission_path'])
print(f"\n📊 SUBMISSION FILE VALIDATION:")
print("="*50)
print(f"✓ Shape: {submission_check.shape}")
print(f"✓ Columns: {list(submission_check.columns)}")
print(f"✓ Data types: {submission_check.dtypes.to_dict()}")
print(f"✓ No missing values: {submission_check.isnull().sum().sum() == 0}")
print(f"✓ All predictions non-negative: {(submission_check['predicted_count'] >= 0).all()}")

print(f"\n🎯 PROJECT COMPLETION STATUS:")
print("="*50)
print("✅ Dataset Analysis - COMPLETED")
print("✅ Model Architecture Design - COMPLETED") 
print("✅ Data Preprocessing Pipeline - COMPLETED")
print("✅ Model Training & Validation - COMPLETED")
print("✅ Performance Evaluation - COMPLETED")
print("✅ Test Set Predictions - COMPLETED")
print("✅ Submission File Generation - COMPLETED")
print("✅ Documentation & Visualization - COMPLETED")